# Algorithmic Methods of Data Mining

## Homework 3
###  Andrea Aiello, Daniele Mocavini, Mani Niharika Rachuri

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict
from os import listdir
from os.path import isfile, join
import numpy
import statistics
from operator import itemgetter
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.corpus import stopwords
import re
from collections import Counter
import json

## Data Collection

In [ ]:
# First identify and save the folder where we have stored all the html files

mypath = "C:/Users/danie/OneDrive/Università/Data Science/Algorithmics (Aris-Yoannis)/Homeworks/Homework 3/aris database/lyrics_collection/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# Now we can parse the pages and take for each all the information we need, like artist, title, url and the song 

song={}
song_list=[]
artists_name_list=[]
d = defaultdict(list)
complete_dictionary=defaultdict(list)
stringa="https://www.azlyrics.com/"
for i in onlyfiles:
    file = open("C:/Users/danie/OneDrive/Università/Data Science/Algorithmics (Aris-Yoannis)/Homeworks/Homework 3/aris database/lyrics_collection/"+ i, "r",encoding="utf8")
    soup = BeautifulSoup(file, 'lxml')
    artist = soup.findAll("span",{ "class" : "item-header-color" })
    title = soup.findAll("h1")
    lyric = soup.findAll("div", { "class" : "dn" })
    try:
        soup1 = BeautifulSoup(str(lyric[0]),'lxml')
        l = soup1.text
        t = str(title[0])
        T = t[4:-12]
        a = str(artist[2])
        A = a[32:-14]
        d[A].append(T)
        song_list.append(l)
        artists_name_list.append(A)
        url=stringa+"lyrics/"+A.replace(" ","").lower()+"/"+T.replace(" ","").lower()+".html"
        complete_dictionary[A,T].append([url, A, T, l])
        song[T]=l
        
            
    except:
        pass

In [ ]:
# For make a local copy of the lists and the dictionaries we created we choce to use numpy for his speed

numpy.save("autori e canzoni",d)
numpy.save("D:/Download/solotesti",song_list)
numpy.save("nomiartisti",artists_name_list)
numpy.save("testi",song)
numpy.save("dizionario completo", complete_dictionary)

In [ ]:
# Load the local copy

d=numpy.load("autori e canzoni.npy").item()

In [ ]:
song_list=numpy.load("D:/Download/solotesti.npy")

In [ ]:
artists_name=numpy.load("nomiartisti.npy")

In [ ]:
song=numpy.load("testi.npy").item()

In [ ]:
dictsongs=numpy.load("dizionario completo.npy").item()

In [ ]:
# Convert orderdict to dictionary and artist_name and song_list to a list

d=dict(d)
artists_name=list(artists_name)


In [ ]:
song_list=list(song_list)

In [ ]:
# Store the parsed songs as documents in MongoDB database, one document per song,
# using MongoDB Hosting: Database-as-a-Service by mLab

dict_of_songs = [] #this will contain a list of dictionaries
for [x,y] in dictsongs.keys():
    #print(dictsongs[x,y])
    a = dictsongs[x,y]
    x=[]
    x.append(a[0][0]) #url
    x.append(a[0][1]) #artist
    x.append(a[0][2]) #song name
    x.append(a[0][3]) #lyrics
    tempdict = {"url": x[0],
                "artist": x[1],
                "song-name": x[2],
                "lyrics": x[3]
            }
    dict_of_songs.append(tempdict)

In [ ]:
params = {'apiKey': 'jJFk7bsNFjagIf9nvxRQzq4AhVot1kkK'}
dbname = 'prova' #my db name in mlab

collection = 'Algorithmic_Methods_of_Data_Mining_Hw3' #new collection created in mlab
url = 'https://api.mlab.com/api/1/databases/' + dbname + '/collections/' + collection
headers = {'content-type': 'application/json'}
data = json.dumps(dict_of_songs[6:20])
response = requests.post(url, data=data, params=params, headers=headers)

# Song Statistics

## Identify Artist with most songs and create a histogram of the number of songs per Artist


In [ ]:
# This first function find the number of song for each artist

def most_song (artist):
    freq=[]
    for i in artist:
        a=(i,len((artist[i])))
        freq.append(a)
    return (freq)

In [ ]:
# With a small change we can calculate also the mean

def most_song_mean (artist):
    freq=[]
    for i in artist:
        a=len(artist[i])
        freq.append(a)
    return ("mean", statistics.mean(freq))

In [ ]:
# And also the median

def most_song_median (artist):
    freq=[]
    for i in artist:
        a=len(artist[i])
        freq.append(a)
    return ("median", statistics.median(freq))

In [ ]:
# Find the number of song for each artist

a=most_song(d)

In [ ]:
# finds the mean of the songs written by each artist

med=most_song_mean(d)
print(med)

In [ ]:
# finds the median of the songs written by each artist

median=most_song_median(d)
print(median)

In [ ]:
# Print the histogram of the number of songs for artist

df = pd.DataFrame(a, columns=['Artist Name', 'frequency'])
frame1=df.plot(kind='bar', x='Artist Name')
frame1.axes.get_xaxis().set_visible(False)
plt.show()

In [ ]:
# We can also create an histogram ordered by number of songs written

ordered_list=sorted(a,key=lambda x: x[1], reverse=True)
df = pd.DataFrame(ordered_list, columns=['Artist Name', 'frequency'])
frame1=df.plot(kind='bar', x='Artist Name')
frame1.axes.get_xaxis().set_visible(False)

plt.annotate('Mean', xy=(0,71.76514522821577), xytext=(200, 200),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )
plt.annotate('Median', xy=(0,45), xytext=(500, 200),
            arrowprops=dict(facecolor='orange', shrink=0.05),
            )
plt.show()


In [ ]:
# We can also find the distribution

more_200 = ("More then 200",sum(Counter([t[1] for t in ordered_list if t[1] > 200]).values()))
equal200=("Equal 200",sum(Counter([t[1] for t in ordered_list if t[1] == 200]).values()))
a_150_to_200 = ("From 150 to 200",sum(Counter([t[1] for t in ordered_list if t[1] >= 100 and t[1] < 200 ]).values()))
a_50_to_100 = ("From 50 to 100",sum(Counter([t[1] for t in ordered_list if t[1] >= 50 and t[1] < 100 ]).values()))
a_25_to_50 = ("From 25 to 50",sum(Counter([t[1] for t in ordered_list if t[1] >= 25 and t[1] < 50 ]).values()))
less_20 = ("Less then 25",sum(Counter([t[1] for t in ordered_list if t[1]  < 25]).values()))
distribution=[more_200,equal200,a_150_to_200,a_50_to_100,a_25_to_50,less_20]
df = pd.DataFrame(distribution, columns=['Distribution', 'frequency'])
frame1=df.plot(kind='bar', x='Distribution')
plt.show()

In [ ]:
# Inside Less then 25 group we have the following situation

a_15_to_25 = ("From 15 to 20",sum(Counter([t[1] for t in ordered_list if t[1] >= 15 and t[1] < 25]).values()))
a_10_to_15 = ("From 10 to 15",sum(Counter([t[1] for t in ordered_list if t[1] >= 10 and t[1] < 15]).values()))
a_5_to_10 = ("From 5 to 10",sum(Counter([t[1] for t in ordered_list if t[1] >= 5 and t[1] < 10]).values()))
less_5 = ("Less then 5",sum(Counter([t[1] for t in ordered_list if t[1]  < 5]).values()))
equal2=("Equal 2",sum(Counter([t[1] for t in ordered_list if t[1] == 2]).values()))
equal1=("Equal 1",sum(Counter([t[1] for t in ordered_list if t[1] == 1]).values()))
distribution=[a_15_to_25,a_10_to_15,a_5_to_10,less_5,equal1,equal2]
df = pd.DataFrame(distribution, columns=['Distribution', 'frequency'])
frame1=df.plot(kind='bar', x='Distribution')
plt.show()

In [ ]:
# Find who wrote more songs

print(ordered_list[0])

In [ ]:
# we can also find the top 10 artist who have written more songs

print(ordered_list[0:10])

In [ ]:
# And also the first 25

print(ordered_list[0:25])

Looking at the data we can first see the difference between mean (about 72) and median (45) so we can suppose to have some anomalous values.
We have in fact that 501 Artist wrote less then 25 songs (of which 65 wrote just 1 song) ed just 37 wrote more then 200 songs, without these excesses we have that 191 artist wrote from 150 to 200 songs (262 artist if we include also the number 200 songs).

In the top 10 most productive artist (excluding "Various Artist") we have just 1 band (Rolling Stones) and also looking at the first 25 we found just 1 other bad (U2), perhaps band's members have more frequently opposing views on songs and for this reason they write less or they melt.

Interestingly also note how in the top 10 most productive artist (excluding "Various Artist") we have even 5 rapper.

In the top 10 we also have 2 deceased artists (David Bowie and Frank Sinatra), searching for each the years active we have

- Wiz Khalifa: 13 years
- Rolling stone: 55 years
- Snoop Dogg: 25 years
- Chris Brown: 12 years
- Elton John: 53 years
- Frank Sinatra: 63 years
- Dolly Parton: 58 years
- Eminem: 25 years
- David Bowie: 54 years
- Lil Wayne: 20 years


So the new ranking based on the number of songs written and years of activity is

- Chris Brown: 17.25
- Wiz Khalifa: 15.76
- Lil Wayne: 10.55
- Eminem: 8.40
- Snoop Dogg: 8.24

- Rolling stone: 7.74
- Elton John: 3.92
- David Bowie: 3.90
- Dolly Parton: 3.60
- Frank Sinatra: 3.13

And again we can observe how rappers write many more songs than authors of other types of music 


## Identify the 20 most popular words (exclude stopwords) and comment

In [ ]:
# Write a new txt files with songs without the most common stopwords of 17 different lenguage
# we have also edited the "english" file adding the the following words:
# "dont", "cant", "youre", "aint".
# we have also decided to add the condition len(word) >=3 to delete other non-included stopwords
# We chose to create a local copy of the new "song file" so that it can be analyzed at any time

with open('outputFile.txt','w',encoding="utf8") as outFile:
    for line in song_list:
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split() 
            if len(word) >=3 and word not in stopwords.words('arabic')
                       and word not in stopwords.words('danish')
                       and word not in stopwords.words('dutch')
                       and word not in stopwords.words('english')
                       and word not in stopwords.words('finnish')
                       and word not in stopwords.words('french')
                       and word not in stopwords.words('german')
                       and word not in stopwords.words('hungarian')
                       and word not in stopwords.words('italian')
                       and word not in stopwords.words('kazakh')
                       and word not in stopwords.words('norwegian')
                       and word not in stopwords.words('portuguese')
                       and word not in stopwords.words('romanian')
                       and word not in stopwords.words('russian')
                       and word not in stopwords.words('spanish')
                       and word not in stopwords.words('swedish')
                       and word not in stopwords.words('turkish')]), file=outFile)

In [ ]:
# Now we can read the output file and found the most common words

with open('outputFile.txt',encoding="utf8") as f:
    passage = f.read()

words = re.findall(r'\w+', passage)
common=Counter(words).most_common(20)
print(common)

we found that the most used word is love, 

## Identify the 10 most common singer names (e.g, “Alice,” “Bob,” “Frank”) and see whether singers whose name is the same tend to publish more songs than others


In [ ]:
# Create a nested list with name and suppose that the name of the artist is always the first word 

wo=[]
for i in artists_name:
    words = re.findall(r'\w+', i)
    wo.append(words)
lst = [item[0] for item in wo]

In [ ]:
# Print the 10 most common name

lst2=Counter(lst).most_common(10)
print(lst2)

In [ ]:
# We have also found "The", and we need to delete it and found again the most common

lst2=Counter(lst).most_common(11)
use=[x[0] for x in lst2]
use.remove('The')
print(use)

In [ ]:
# Remember the top 25 more productive artist

aba=ordered_list[0:25]
print(aba)

In [ ]:
# Select name and surname, create the frequence and concatenate

woo=[]
for i in [x[0] for x in aba]:
    words = (re.findall(r'\w+', i))
    woo.append(words)

test=[x[1] for x in aba]

a=[x for x in zip(woo, test)]

In [ ]:
# Find if there are 1 or more most common artist name in the top 25 most productive artist

for i in use:
    print((i,[item for item in a if item[0][0] == i]))

We found that 5 of the most common name ("John", "Bob", "Neil", "Chris" and "Paul") is in the top 25 of the most productive artist:

- John Denver: 204
- Bob Dylan: 205
- Neil Young: 203
- Chris Brown: 207
- Paul McCartney: 205

## Create a histogram of song lengths 

In [ ]:
# Found the title for each song
titolo=[]
for i in song:
    titolo.append(i)

In [ ]:
# Count number of words in each songs

num_words = [len(sentence.split()) for sentence in [value for value in song.values()]]

In [ ]:
# Concatenate
unitaa=[x for x in zip(titolo, num_words)]

In [ ]:
# Print the histogram

df = pd.DataFrame(unitaa, columns=['Artist Name', 'frequency'])
frame1=df.plot(kind='bar', x='Artist Name')
frame1.axes.get_xaxis().set_visible(False)
plt.show()

In [ ]:
# order the list by song's length

ordered_unitaa=sorted(unitaa,key=lambda x: x[1], reverse=True)

In [ ]:
# We can also find the distribution

more_2000 = ("More then 2000 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] > 2000]).values()))
a_1000_to_2000 = ("From 1000 to 2000 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] >= 1000 and t[1] < 2000 ]).values()))
a_500_to_1000 = ("From 500 to 1000 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] >= 500 and t[1] < 1000 ]).values()))
a_250_to_500 = ("From 250 to 500 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] >= 250 and t[1] < 500 ]).values()))
a_100_to_250 = ("From 100 to 250 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] >= 100 and t[1] < 250 ]).values()))
a_50_to_100 = ("From 50 to 100 words",sum(Counter([t[1] for t in ordered_unitaa if t[1] >= 50 and t[1] < 100 ]).values()))
less_50 = ("Less then 50 words",sum(Counter([t[1] for t in ordered_list if t[1]  < 50]).values()))
distribution=[more_2000,a_1000_to_2000,a_500_to_1000,a_250_to_500,a_100_to_250,a_50_to_100,less_50]
df = pd.DataFrame(distribution, columns=['Distribution', 'frequency'])
frame1=df.plot(kind='bar', x='Distribution')
plt.show()
print("This is the distribution", distribution)